## Frame

In [235]:
using Gridap
using Plots
using GridapMakie
using CairoMakie 
using GLMakie
using Gridap.Geometry
using GridapGmsh

In [236]:
# Define known parameters/functions
E = 200e9 # Young's modulus (Pa)
A = 0.01 # Cross-sectional area (m²)
EA = E*A

f(x) = 1000.0 # N/m
T = 500e6 # N (tensile force acting on the middle and top element)

model = GmshDiscreteModel("msh_files/OFFSHORE_JACKET.msh")

Info    : Reading 'msh_files/OFFSHORE_JACKET.msh'...
Info    : 19 entities
Info    : 955 nodes
Info    : 963 elements
Info    : Done reading 'msh_files/OFFSHORE_JACKET.msh'


UnstructuredDiscreteModel()

In [237]:
# Set up the figure and axis 
initial_tags = get_tag_entities(labels)[end]
labels = get_face_labeling(model)
tag_from_names = get_tag_from_name(labels)

Dict{String, Int64} with 6 entries:
  "Point of application for force" => 2
  "Base"                           => 3
  "Middle"                         => 4
  "Diagonals"                      => 6
  "Top"                            => 5
  "Base legs"                      => 1

In [238]:
# Set up the figure and axis (3D if model is 3D)
fig = Figure(resolution=(800, 600))
ax = Axis(fig[1, 1], title="Domain with Boundary Tags", xlabel="x", ylabel="y")

for intgr in length(tag_from_names)
    CairoMakie.wireframe!(ax, Triangulation(model))
end

CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Base legs"))
CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Point of application for force"))

fig

In [239]:
add_tag_from_tags!(labels, "Dirichlet", [1]) # Our Dirichlet is the base (fixed displacement) (tag = 1)
add_tag_from_tags!(labels, "Neumann", [2]); # Our Neumann BC is at the point of application for force (tag = 2)

In [240]:
Ω = Triangulation(model)
Γ_D = Boundary(Ω,tags="Dirichlet")
Γ_N = Boundary(Ω,tags="Neumann")

CompositeTriangulation()

In [241]:
order = 1
reffe = ReferenceFE(lagrangian, Float64, order) # We create a Reference Finite Element (it serves as a basis function on one single triangulation element (puzzle piece of the entire puzzle))
V = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags="Dirichlet")
U = TrialFESpace(V, 0.0)

TrialFESpace()

In [242]:
degree = 2*order 
dΩ = Measure(Ω,degree) # Numerical integration within the domain Ω using second-order Gaussian quadrature rule
dΓ_N = Measure(Γ_N,degree) # Numerical integration along the Neumann boundary Γ_D using second-order Gaussian quadrature rule

GenericMeasure()

In [243]:
# Weak form
a(u, v) = ∫(EA * ∇(u) ⋅ ∇(v))dΩ
l(v) = ∫(f * v)dΩ + ∫(T * v)dΓ_N

l (generic function with 1 method)

In [244]:
# Solve the problem
op = AffineFEOperator(a, l, U, V)
uh = solve(op)

writevtk(Ω,"rod_solution",cellfields=["uh"=>uh])

(["rod_solution.vtu"],)